## Informed Search Algorithms
### Artificial Intelligence 1: week 3

## This week
- Recap: uninformed search algorithms for decision problems

- Heuristic Quality Functions

- General framework for informed search
  - Search guided by cost/quality function  
    Without backtracking:  Hill Climbing (local search)  
    With backtracking: Best-First Search
  - Search taking into account cost of steps taken  
    Dijkstra (path-finding),  A* 

    
- Example applications:
    - inside Machine Learning algorithms
    - path-finding
    - optimisation
    
- Strengths and weaknesses to take into account  
  when selecting a search algorithm to apply to a problem
  

## Recap: common framework
<img src="./figures/generate-and-test-framework.png" >




## Recap: Decision problems:
- yes / no answer (needle in a haystack), **so nothing to guide search**
- e.g. logic puzzles, combination lock, 
- but lots of real world examples too

Depth-First and Breadth-first search
- up and down the tree or across side to side

In [ ]:
import notshared
from notshared import DepthFirstSearch
from notshared import BreadthFirstSearch
from notshared import LocalSearch
from notshared import BestFirstSearch
from notshared import AStarSearch
from notshared import DijkstraSearch
from maze import Maze

### Simple function to demonstrate algorithms running on toy maze

In [ ]:
from singlemembersearch import SingleMemberSearch


def test_on_maze(algorithm: SingleMemberSearch):
    mymaze = Maze(mazefile="maze.txt")
    mysearch = algorithm(mymaze, constructive=True, max_attempts=1500)
    found = mysearch.run_search()
    if found:
        print(
            f"search using {mysearch.__str__()} algorithm successful after {mysearch.trials} attempts"
            f" length of path is {len(mysearch.result)} moves."
        )
    else:
        print("solution not found in time allowed")
        print(mysearch.runlog)
    del mymaze

### Depth-First then Breadth-First Search  Behaviour

In [ ]:
test_on_maze(DepthFirstSearch)

In [ ]:
# And now for breadth-first
test_on_maze(BreadthFirstSearch)

## How could we make those better?<img src="figures/timer.png" style="float:right" width = 15%>


Breadth/depth-first generate nodes to test:
-  based on the shape of the tree,
- ignoring  how good the solutions are,  
- or how close they might be to the goal state.

we say they are “blind” or “uninformed”.

More efficient approach is to incorporate information about how close you are to the solution

USE ANYTHING YOU HAVE TO HAND if it helps you avoid constraints!
<img src="figures/multitool.png" style="float:right" width = 20%>

## Quality Functions
Natural for some problems, e.g.:
- **Model Building**: error rate of model on training set,
- **Optimisation**: Distance, cost, payoff
- **Prediction**: error rate of model in real world…

Often more than one - hence **“heuristic”** (rule of thumb)  
Some may take more effort to calculate
- simulations run at different fidelity,
- User studies with different sized groups
- Different ways of estimating machine learning  model accuracy


## Estimated Quality Measures
For other problems we can define a
	“heuristic evaluation function” , h(n)  for each node n:
 - provides information to **guide**  *informed search*.
 - **estimates** how far a node is from the goal state,

Comparing two nodes m  and n 
- h(m) < h(n) implies m is closer to the goal.
- So typically we look to **minimise** the function.

Also known as a ...  
- cost function  
-  quality function,  
-  ‘score’,  
-  ‘fitness’  (to be maximised)


## Choosing Heuristic Functions
- Should be quick to calculate
 
- Might simplify or ignore constraints (especially 'soft' ones)
 
- The more different levels the better (provide more information to search)
 
- Should be "optimistic" (underestimate distance/cost)
  - e.g. training set accuracy underestimates error on unseen data

### Example: Two landscapes created from the same function
### with quality calculated at different levels of precision
![integer vs 1 decimal place](./figures/2landscapes.png)

## Adding heuristic functions to our generate and test code

Minor change to pseudocode we had for depth and breadth first:

Now our Evaluate() function gives some idea of quality instead of just feasibility

So all we need to change is the method  ```SelectAndMoveFromOpenList(algorithm_name)```

Typically to return the one with the highest quality
 - Could do the same with uninformed search using quality = age
 - but we get that for 'free' by using a list,     
    and always adding new things to the end, 

The function may also do some memory management of the open list.
- **next year in AI2 you'll see how extending this gives rise to some more complicated search algorithms**

## Simple example: Hill Climbing local search (minimising)

Loop through open list and pick the one with lowest quality/cost <=> increasing distance_to_goal

**Doesn’t allow back-tracking**
  - Discard rest of open list after copying the chosen one
  - so next each time around the open list just contains the neighbours of the current working candidate


**Only accept improving moves**
 - stop if the best neighbour is not an improvement on the current working candidate
 - i.e. you are on local optima or a plateau
 - even if goal / global optimum not reached

greedy/steepest ascent variants:
 - Examine all child nodes and follow first / best  improvement.

**Quick but gets stuck in local optima**


## Pseudocode for function SelectAndMoveFromOpenList in Local Search
### This assumes the search process maintains track of *bestSoFar*
<div style="background:#F0FFFF;font-size:18pt">
<p style="color:darkred;font-size:18pt;margin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="font-size:18pt;margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd>bestChild &larr; <b>GetMemberWithHighestQuality</b>(openList)</dd>
    <dd> <b>EMPTY</b>(openlist)&nbsp;&nbsp;&nbsp;&nbsp;<span style="background:pink">This prevents backtracking</span></dd>
    <dd>  <b>IF</b> BetterThan(bestChild, bestSoFar) <b>THEN</b> <br>
        &nbsp;&nbsp;&nbsp;&nbsp;bestSoFar &larr; bestChild <br>
        &nbsp;&nbsp;&nbsp;&nbsp;RETURN bestChild </dd>
    <dd> <b>ELSE</b> <br>&nbsp;&nbsp;&nbsp;&nbsp; RETURN None</dd>
</dl>
</div>    

In [ ]:
test_on_maze(LocalSearch)

## Hill Climbing can get stuck even on our simple example! <img src="figures/hillclimbing-tree.png" style="float:right" width=50%>

- But is fast
- and you can always restart it from another place

Examples:
- Optimisation:
  - timetabling
- Machine Learning
  - most decision tree algorirthms
   - greedy rule induction     
- "Stochastic Gradient Descent" (aka backprop)
  for training neural networks


## Best First Search <img src="figures/best-first-tree.png" style="float:right" width = 50%>
Like hill-climbing is driven by quality  
**but keeps unused nodes in the open list**

At every iteration:
- finds and returns the member in the open list with the best quality,  
- leavves the rest in the open list
- i.e. doesn’t removed unexplored nodes  
  This adds  backtracking

*Tends* to produce shorter paths than depth- or breadth first search


## Pseudocode for function SelectAndMoveFromOpenList in Best-First Search

<div style="background:#F0FFFF;font-size:18pt">
<p style="color:darkred;font-size:18pt;margin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="font-size:18pt;margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd>bestChild &larr; <b>GetMemberWithHighestQuality</b>(openList)</dd>
    <dd> RETURN bestChild&nbsp;&nbsp;&nbsp;&nbsp;<span style="background:pink">Best-First keeps the openlist to allow backtracking</span></dd>
</dl>
</div>   

In [ ]:
test_on_maze(BestFirstSearch)

## Quiz Questions:
- Hill-Climbers can get stuck in local optima (True:False)
- The local optima a hill-climber finds, depends on where it starts (True:False)
- Which of these might help local search?
Vote for as many as you think will
  - Multiple runs from random starting places
  - Multiple runs, start each one by making random changes to  the last local optimum
  - One  run, changing the move operator everytime you reach a local optimum
  - Sometimes accepting worsening moves


# Pause

## Taking into account the cost of reaching a solution
 <img src="figures/balanced_plates.jpg" style="float:right">


E.g. planning 
- Routes to avoid toll roads (cost)  
- Routes to avoid  built-up areas (air pollution)
- the path of a manipulator to reduce number of moves
- the path of a manipulator avoiding sudden changes of direction  

or "Occcam's Razor": the simplest explanation (ML model) is usually the best

## A* : **guaranteed** shortest/least cost paths <img src="figures/optimal.png" style="float:right" width = 25%>
Adds cost to Best-First to find optima
 - Shortest path / least complex model,

Sorts the list of unexplored nodes by f(node):
- f(node) = g(node) + h(node),  
  h(node) =  estimated distance to goal (heuristic).  
  g(node) = cost of reaching that node.

So you can stop looking as soon as  you know that   
g(node) > best_so_far for all remaining nodes

**A* is complete and optimal as long as h(node) is an underestimate**

A* is used for : path-finding (especially NPCs in games),   query optimisation, …


## Pseudocode for function SelectAndMoveFromOpenList in AStar Search

<div style="background:#F0FFFF;font-size:18pt">
<p style="color:darkred;font-size:18pt;margin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="font-size:18pt;margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd><span style="background:pink">AStar picks using sum of quality +cost</span></dd>
    <dd>bestChild &larr; <b>GetMemberWithHighestCombinedScore</b>(openList)</dd>
    <dd> RETURN bestChild&nbsp;&nbsp;&nbsp;&nbsp;</dd>
</dl>
</div>   
<div style="background:white"> <h3>Note that</h3><ul>
    <li>This is just like best-first with a modified selection.</li>
    <li> To make more efficient you can track <i>bestSoFar</i> and modify <b>UpdateWorkingMemory()</b><br>
        so it doesn't put things on the openlist if depth > bestSoFar </li></ul>  </div> 

In [ ]:
test_on_maze(AStarSearch)

## A* example <img src="figures/Astar-tree.png" style = "float:right" width = 40%>
 We show the cost as a second number in each node – in this case just the depth

## What does "optimality" mean for A* ?
Finds node which satisfies the goal criteria

If there is more than one of these, it finds the one with the least cost 

How else could we interpret this?
What might be desirable?




# Dijktra's algorithm
<img src = "figures/dijkstra.gif" style = "float:right" width = 40%>

Designed for use in tracing routes between points in an weighted undirected graph.
- "weighted" means there are cost/distances on each link   
  (edge in the graph)  
  e.g. tolls/ different terrains, ...
- "undirected" means you can traverse an edge  
  in either direction ( no one-way roads)
- copes with cycloes/loops by continuously updating distance estimates

Finds the single shortest path between two points.  
Most used for path-finding – a *lot* in games

Like A* but ignores heuristic cost
h(n) = 0 for all n

"Dijkstra Animation" by Ibmua - Work by uploader.. Licensed under Public Domain via Commons - https://commons.wikimedia.org/wiki/File:Dijkstra_Animation.gif#/media/File:Dijkstra_Animation.gif


## What does this mean in terms of our existing pseudocode?

Criteria for picking working candidate from open list
- closest distance to start
- **going through nodes we know about**
- so these need updating

Just like the normal quality function, sometimes we might need to approximate that
- if we just use path length it ends up being the same as Breadth Frst Search 

## Pseudocode for function SelectAndMoveFromOpenList in Dijkstra's Search Algorithm

<div style="background:#F0FFFF;font-size:18pt">
<p style="color:darkred;font-size:18pt;margin-bottom:0pt"><em>SelectAndMoveFromOpenList</em></p>
<dl style="font-size:18pt;margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>IF</b> IsEmpty( open_list) <b>THEN</b> </dt>
    <dd> RETURN None</dd>
    <dt> &nbsp;&nbsp;&nbsp;<b>ELSE</b></dt>
    <dd><b>UpdateDistanceEstimates</b>(OpenList)<br>
    <dd>bestChild &larr; <b>GetMemberWithLowestDistance</b>(openList)</dd>
    <dd> RETURN bestChild&nbsp;&nbsp;&nbsp;&nbsp;</dd>
</dl>
</div>   

In [ ]:
test_on_maze(DijkstraSearch)

## Really good description and discussion of how different algorithms can be used for path-finding

http://www.redblobgames.com/pathfinding/a-star/introduction.html
    
    

## Quiz Questions
For a ‘decision’ problem, which of these   would be appropriate ?

For an exam timetabling problem which of these   would be appropriate ?

For a npc planning a path to chase someone in a game, which of these   would be appropriate ?

For organising daily delivery schedules, which of these   would be appropriate ?

- Depth-first
- Breadth-first
- Hill-Climbing
- Best-First
- A*
- Dijkstra


## Strengths and weaknesses of different approaches

You should construct a tables listing whether different algorithms possess these features

- Completeness?
  - will get stuck in local optima?
  - avoids getting stuck in loops by design?
  - can (sometimes) be adapted to avoid being stuck in loops? 
- Efficiency, e.g.
  - Can make more efficient by incorporating information about solution quality?
  - Limited storage overheads
- Optimality:
  - Guaranteed to find a solution if one exists?
  - Can find least-cost/short/least complex solutions?
  
 And look at the reading lists, examples of applications using search algorithms  (sat navs, machine learning, ...) to understand why they make the choices and trade-offs they do

## Summary of search topic:

You need to know about and recognise:
- Common framework
- Depth and Best first search when there is no quality function
- Characteristics of a good heuristic quality function
- Simple Hill Climber
- Best first
- A*
- Dijkstra’s Algorithm

You should be able to answer questions about
- How to implement different strategies within a common framework
- Choosing an appropriate search strategy for a problem:
  - Do you have a way of assigning quality?
  - What are your trade-offs for time vs storage vs optimality?
  - Can ‘good-enough’ be ok?


# Extras: Optimising TSP

In [ ]:
import random
import numpy as np
import math
from matplotlib i
from problem import Problem

# place cities in random positions


class TSP(Problem):
    """Class for TSP problems
    Each instancce is randomly generated
    """

    def __init__(self, num_cities=10):
        """gnerates a new random indiex uniformly on a 100x100 grid"""
        self.num_decisions = num_cities
        # value set is just the id's of the cities
        self.value_set = list(range(0, num_cities))

        # random generation of city locations
        self.cities = [random.sample(range(100), 2) for x in range(num_cities)]
        # model is just the magtrix of inter-city distances
        self.model = self.get_distances()
        self.plot_cities()

    def get_distances(self):
        num_cities = len(self.cities)
        distances = np.zeros((num_cities, num_cities))
        for row in range(num_cities):
            for col in range(num_cities):
                if row != col:
                    xdist = self.cities[row][0] - self.cities[col][0]
                    ydist = self.cities[row][1] - self.cities[col][1]
                    distances[row][col] = math.sqrt(xdist * xdist + ydist * ydist)
        return distances

    def plot_cities(self):
        fig, ax = plt.subplots()
        for i in self.value_set:
            ax.plot(self.cities[i][0], self.cities[i][1], "Xb")
        modelstrings = np.array(
            ["%.2f" % x for x in self.model.reshape(self.model.size)]
        )
        modelstrings = modelstrings.reshape(self.model.shape)
        ax.table(cellText=modelstrings, loc="right", bbox=[1.1, 0, 1, 1])
        plt.show()

    def show_tour(self, tour: list):
        plt.plot(self.cities[start][0], self.cities[start][1], "or", markersize=12)
        n = self.num_decisions
        plt.plot(
            [self.cities[tour[i % n]][0] for i in range(n + 1)],
            [self.cities[tour[i % n]][1] for i in range(n + 1)],
            "Xb-",
        )
        plt.show()

    def evaluate(self, tour: list) -> int:
        """simple sum of distance between places in tour
        wrapping around to the start"""

        n = len(tour)  # number of cities visited
        # two sanity checks - not relevant for partial tours
        # errstr= f"len mismatch {n} vs. {self.num_decisions}"
        # assert n== self.num_decisions,errstr

        # every value in tour must be a unique valid index
        for val in tour:
            if val not in self.value_set:
                return -1, "invalid city number in tour"
        uniqs = np.unique(np.array(tour), return_counts=False)
        if len(uniqs) != n:
            return -1, "non-unique value in tour"

        distance = 0
        # simple loop from place to place in tour
        for pos in range(n):
            # using modulus so that at the end we go back to the start
            nextpos = (pos + 1) % n
            current_place = tour[pos]
            next_place = tour[nextpos]
            dist = self.model[current_place][next_place]
            distance += int(dist)

        # now penalise tours that don't visit every city
        penalty = 1000 * (self.num_decisions - n)
        return distance + penalty, ""

In [ ]:
n = 10
mytsp = TSP(num_cities=n)

In [ ]:
for start in range(5):
    mysearch = LocalSearch(mytsp, constructive=True, minimise=True, max_attempts=1000)
    # seed first city
    mysearch.open_list[0].variable_values.append(start)
    mysearch.run_search()
    # print(mysearch.runlog)
    last_solution = mysearch.closed_list[-1]
    print(
        f"last solution has quality {last_solution.quality} for tour {last_solution.variable_values}."
    )
    mytsp.show_tour(last_solution.variable_values)